In [1]:
import sys
sys.path.append('./../')
import tensorflow as tf
print(tf.__version__)

# Helper libraries
from tensorflow import keras
from keras import mixed_precision
import numpy as np
import time
import maxine
import sys
from maxine.layers import SoftMaxMin_Pruned_v02
import keras.backend as k
import pruning_tool_v2
from maxine.tools import pruning_tool_MLP_v3
from maxine.tools import pruning_tool_MLP_v4_1
from maxine.tools import pruning_tool_MLP_v4_2
import tensorflow_model_optimization as tfmot
from keras import regularizers
from keras.callbacks import EarlyStopping
from scipy.io import loadmat

2023-12-29 10:29:42.092521: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-29 10:29:42.965430: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.0


In [2]:
# LOAD DATA
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# convert to float and normalize
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

# Validation data
x_val = x_train[-5000:]
y_val = y_train[-5000:]

x_train = x_train[:-5000]
y_train = y_train[:-5000]

In [3]:
baselineModel = tf.keras.models.load_model('../models/paper/MLP_FASHION_300_100_v2')

2023-12-21 11:36:30.206781: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2031] Ignoring visible gpu device (device: 2, name: NVIDIA GeForce GT 1030, pci bus id: 0000:84:00.0, compute capability: 6.1) with core count: 3. The minimum required count is 8. You can adjust this requirement with the env var TF_MIN_GPU_MULTIPROCESSOR_COUNT.
2023-12-21 11:36:31.520715: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23664 MB memory:  -> device: 0, name: Tesla P40, pci bus id: 0000:03:00.0, compute capability: 6.1
2023-12-21 11:36:31.521416: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 23664 MB memory:  -> device: 1, name: Tesla P40, pci bus id: 0000:04:00.0, compute capability: 6.1


In [4]:
baselineModel.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3119 - accuracy: 0.8992


[0.31187891960144043, 0.8992000222206116]

In [5]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, weight_decay=0.0005)

In [6]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

In [7]:
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=0, optimizer_config=tf.optimizers.serialize(optimizer), extreme=False)

Training morphological model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 300)              235200    
                                                                 
 dense (Dense)               (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30


2023-12-19 12:08:33.867401: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x233ff4a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-19 12:08:33.867472: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla P40, Compute Capability 6.1
2023-12-19 12:08:33.867489: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla P40, Compute Capability 6.1
2023-12-19 12:08:33.882405: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-19 12:08:34.085113: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-12-19 12:08:34.202646: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-19 12:08:34.404646: I ./tensorflow/compiler/jit/device_compiler.h:186] Compil

1719/1719 - 11s - loss: 0.9001 - accuracy: 0.7169 - val_loss: 0.5208 - val_accuracy: 0.8136 - 11s/epoch - 7ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4795 - accuracy: 0.8346 - val_loss: 0.4422 - val_accuracy: 0.8430 - 8s/epoch - 5ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4272 - accuracy: 0.8511 - val_loss: 0.4326 - val_accuracy: 0.8488 - 8s/epoch - 5ms/step
Epoch 4/30
1719/1719 - 8s - loss: 0.3971 - accuracy: 0.8595 - val_loss: 0.3943 - val_accuracy: 0.8568 - 8s/epoch - 5ms/step
Epoch 5/30
1719/1719 - 8s - loss: 0.3751 - accuracy: 0.8685 - val_loss: 0.4022 - val_accuracy: 0.8594 - 8s/epoch - 5ms/step
Epoch 6/30
1719/1719 - 8s - loss: 0.3592 - accuracy: 0.8743 - val_loss: 0.3809 - val_accuracy: 0.8612 - 8s/epoch - 5ms/step
Epoch 7/30
1719/1719 - 8s - loss: 0.3448 - accuracy: 0.8777 - val_loss: 0.3700 - val_accuracy: 0.8680 - 8s/epoch - 4ms/step
Epoch 8/30
1719/1719 - 8s - loss: 0.3341 - accuracy: 0.8826 - val_loss: 0.3711 - val_accuracy: 0.8712 - 8s/epoch - 5ms/step
Epoch 9/30
1719/1

2023-12-19 12:14:19.073800: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:57] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. sequential/modelLayer_1_0/dropout/random_uniform/RandomUniform


430/430 [==============================] - 5s 8ms/step - loss: 0.6452 - accuracy: 0.7687 - val_loss: 0.4349 - val_accuracy: 0.8396
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.4823 - accuracy: 0.8256 - val_loss: 0.4099 - val_accuracy: 0.8430
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.4467 - accuracy: 0.8371 - val_loss: 0.3733 - val_accuracy: 0.8568
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.4284 - accuracy: 0.8440 - val_loss: 0.3792 - val_accuracy: 0.8570
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.4139 - accuracy: 0.8484 - val_loss: 0.3566 - val_accuracy: 0.8712
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.4011 - accuracy: 0.8518 - val_loss: 0.3474 - val_accuracy: 0.8690
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3973 - accuracy: 0.8548 - val_loss: 0.3542 - val_accuracy: 0.8658
Epoch 8/40
430/43

In [8]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 3s 5ms/step - loss: 0.2981 - accuracy: 0.8897 - val_loss: 0.2971 - val_accuracy: 0.8882
Epoch 2/40
430/430 [==============================] - 2s 5ms/step - loss: 0.2906 - accuracy: 0.8919 - val_loss: 0.2942 - val_accuracy: 0.8902
Epoch 3/40
430/430 [==============================] - 2s 5ms/step - loss: 0.2860 - accuracy: 0.8943 - val_loss: 0.2948 - val_accuracy: 0.8876
Epoch 4/40
430/430 [==============================] - 2s 5ms/step - loss: 0.2854 - accuracy: 0.8943 - val_loss: 0.2929 - val_accuracy: 0.8898
Epoch 5/40
430/430 [==============================] - 2s 5ms/step - loss: 0.2847 - accuracy: 0.8948 - val_loss: 0.2950 - val_accuracy: 0.8878
Epoch 6/40
430/430 [==============================] - 2s 5ms/step - loss: 0.2820 - accuracy: 0.8956 - val_loss: 0.2948 - val_accuracy: 0.8890
Epoch 7/40
430/430 [==============================] - 2s 5ms/step - loss: 0.2811 - accuracy: 0.8962 - val_loss: 0.2979 - val_accuracy: 0.8910
Epoch 

[0.307034969329834, 0.8883000016212463]

p = 5

In [7]:
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=5, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

Training morphological model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 300)              235200    
                                                                 
 dense (Dense)               (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30


2023-12-14 10:25:53.283010: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x21acba40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-14 10:25:53.283088: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla P40, Compute Capability 6.1
2023-12-14 10:25:53.283104: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla P40, Compute Capability 6.1
2023-12-14 10:25:53.301489: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-14 10:25:53.514362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-12-14 10:25:53.634565: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-14 10:25:53.832419: I ./tensorflow/compiler/jit/device_compiler.h:186] Compil

1719/1719 - 12s - loss: 0.9003 - accuracy: 0.7183 - val_loss: 0.5230 - val_accuracy: 0.8192 - 12s/epoch - 7ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4838 - accuracy: 0.8324 - val_loss: 0.4639 - val_accuracy: 0.8358 - 8s/epoch - 5ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4279 - accuracy: 0.8509 - val_loss: 0.4221 - val_accuracy: 0.8518 - 8s/epoch - 5ms/step
Epoch 4/30
1719/1719 - 8s - loss: 0.3969 - accuracy: 0.8619 - val_loss: 0.4035 - val_accuracy: 0.8542 - 8s/epoch - 5ms/step
Epoch 5/30
1719/1719 - 8s - loss: 0.3734 - accuracy: 0.8684 - val_loss: 0.3922 - val_accuracy: 0.8550 - 8s/epoch - 5ms/step
Epoch 6/30
1719/1719 - 8s - loss: 0.3574 - accuracy: 0.8748 - val_loss: 0.3839 - val_accuracy: 0.8638 - 8s/epoch - 5ms/step
Epoch 7/30
1719/1719 - 8s - loss: 0.3435 - accuracy: 0.8799 - val_loss: 0.3611 - val_accuracy: 0.8666 - 8s/epoch - 5ms/step
Epoch 8/30
1719/1719 - 8s - loss: 0.3334 - accuracy: 0.8824 - val_loss: 0.3550 - val_accuracy: 0.8674 - 8s/epoch - 5ms/step
Epoch 9/30
1719/1

2023-12-14 10:37:35.555610: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:57] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. sequential/modelLayer_1_0/dropout/random_uniform/RandomUniform


430/430 [==============================] - 6s 9ms/step - loss: 0.8594 - accuracy: 0.6863 - val_loss: 0.5180 - val_accuracy: 0.8114
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.6220 - accuracy: 0.7719 - val_loss: 0.4726 - val_accuracy: 0.8248
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.5819 - accuracy: 0.7842 - val_loss: 0.4764 - val_accuracy: 0.8212
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.5647 - accuracy: 0.7901 - val_loss: 0.4410 - val_accuracy: 0.8312
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.5522 - accuracy: 0.7946 - val_loss: 0.4365 - val_accuracy: 0.8326
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.5419 - accuracy: 0.7993 - val_loss: 0.4244 - val_accuracy: 0.8402
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.5361 - accuracy: 0.8048 - val_loss: 0.4063 - val_accuracy: 0.8446
Epoch 8/40
430/43

In [9]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 3s 5ms/step - loss: 0.6145 - accuracy: 0.7758 - val_loss: 0.4634 - val_accuracy: 0.8294
Epoch 2/40
430/430 [==============================] - 2s 5ms/step - loss: 0.6136 - accuracy: 0.7774 - val_loss: 0.4669 - val_accuracy: 0.8236
Epoch 3/40
430/430 [==============================] - 2s 5ms/step - loss: 0.6079 - accuracy: 0.7798 - val_loss: 0.4586 - val_accuracy: 0.8308
Epoch 4/40
430/430 [==============================] - 2s 5ms/step - loss: 0.6080 - accuracy: 0.7794 - val_loss: 0.4599 - val_accuracy: 0.8304
Epoch 5/40
430/430 [==============================] - 2s 5ms/step - loss: 0.6097 - accuracy: 0.7799 - val_loss: 0.4589 - val_accuracy: 0.8308
Epoch 6/40
430/430 [==============================] - 2s 5ms/step - loss: 0.6047 - accuracy: 0.7788 - val_loss: 0.4580 - val_accuracy: 0.8294
Epoch 7/40
430/430 [==============================] - 2s 5ms/step - loss: 0.6105 - accuracy: 0.7770 - val_loss: 0.4588 - val_accuracy: 0.8302
Epoch 

[0.4832211434841156, 0.8292999863624573]

P = 3%

In [7]:
baselineModel = tf.keras.models.load_model('../models/paper/MLP_FASHION_300_100_v2')
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, nesterov=True)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=5, mlp_epochs_refit=40, p=3, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

Training morphological model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 300)              235200    
                                                                 
 dense (Dense)               (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30


2023-12-21 11:41:33.892036: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x23da7c70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-21 11:41:33.892108: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla P40, Compute Capability 6.1
2023-12-21 11:41:33.892124: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla P40, Compute Capability 6.1
2023-12-21 11:41:33.906348: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-21 11:41:34.106206: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-12-21 11:41:34.226129: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-21 11:41:34.407664: I ./tensorflow/compiler/jit/device_compiler.h:186] Compil

1719/1719 - 11s - loss: 0.9020 - accuracy: 0.7115 - val_loss: 0.5227 - val_accuracy: 0.8212 - 11s/epoch - 7ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4856 - accuracy: 0.8321 - val_loss: 0.4731 - val_accuracy: 0.8292 - 8s/epoch - 4ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4329 - accuracy: 0.8494 - val_loss: 0.4113 - val_accuracy: 0.8540 - 8s/epoch - 5ms/step
Epoch 4/30
1719/1719 - 8s - loss: 0.4005 - accuracy: 0.8595 - val_loss: 0.3955 - val_accuracy: 0.8600 - 8s/epoch - 4ms/step
Epoch 5/30
1719/1719 - 8s - loss: 0.3767 - accuracy: 0.8692 - val_loss: 0.3779 - val_accuracy: 0.8668 - 8s/epoch - 5ms/step
Epoch 6/30
1719/1719 - 8s - loss: 0.3616 - accuracy: 0.8723 - val_loss: 0.3796 - val_accuracy: 0.8606 - 8s/epoch - 4ms/step
Epoch 7/30
1719/1719 - 8s - loss: 0.3458 - accuracy: 0.8792 - val_loss: 0.3616 - val_accuracy: 0.8692 - 8s/epoch - 5ms/step
Epoch 8/30
1719/1719 - 8s - loss: 0.3341 - accuracy: 0.8823 - val_loss: 0.3528 - val_accuracy: 0.8724 - 8s/epoch - 5ms/step
Epoch 9/30
1719/1

2023-12-21 11:47:11.710012: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:57] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. sequential/modelLayer_1_0/dropout/random_uniform/RandomUniform


430/430 [==============================] - 5s 8ms/step - loss: 1.2825 - accuracy: 0.5591 - val_loss: 0.7777 - val_accuracy: 0.7712
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.8884 - accuracy: 0.6963 - val_loss: 0.6662 - val_accuracy: 0.7862
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.7982 - accuracy: 0.7252 - val_loss: 0.6127 - val_accuracy: 0.7978
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.7474 - accuracy: 0.7419 - val_loss: 0.5851 - val_accuracy: 0.8010
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.7176 - accuracy: 0.7517 - val_loss: 0.5618 - val_accuracy: 0.8070
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.6912 - accuracy: 0.7611 - val_loss: 0.5458 - val_accuracy: 0.8098
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.6717 - accuracy: 0.7659 - val_loss: 0.5316 - val_accuracy: 0.8134
Epoch 8/40
430/43

In [8]:
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 3s 5ms/step - loss: 0.7239 - accuracy: 0.7414 - val_loss: 0.5412 - val_accuracy: 0.8094
Epoch 2/40
430/430 [==============================] - 2s 5ms/step - loss: 0.7216 - accuracy: 0.7391 - val_loss: 0.5395 - val_accuracy: 0.8098
Epoch 3/40
430/430 [==============================] - 2s 4ms/step - loss: 0.7203 - accuracy: 0.7415 - val_loss: 0.5383 - val_accuracy: 0.8092
Epoch 4/40
430/430 [==============================] - 2s 5ms/step - loss: 0.7213 - accuracy: 0.7405 - val_loss: 0.5370 - val_accuracy: 0.8102
Epoch 5/40
430/430 [==============================] - 2s 5ms/step - loss: 0.7181 - accuracy: 0.7429 - val_loss: 0.5356 - val_accuracy: 0.8106
Epoch 6/40
430/430 [==============================] - 2s 5ms/step - loss: 0.7140 - accuracy: 0.7440 - val_loss: 0.5337 - val_accuracy: 0.8108
Epoch 7/40
430/430 [==============================] - 2s 5ms/step - loss: 0.7086 - accuracy: 0.7463 - val_loss: 0.5327 - val_accuracy: 0.8108
Epoch 

[0.5330615043640137, 0.8094000220298767]

P = 1%

In [18]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

In [14]:
baselineModel = tf.keras.models.load_model('../models/paper/MLP_FASHION_v3')

In [15]:
baselineModel.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.4393 - accuracy: 0.8978


[0.4392929673194885, 0.8978000283241272]

In [16]:
baselineModel.get_config()

{'name': 'sequential',
 'layers': [{'module': 'keras.layers',
   'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 784),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_input'},
   'registered_name': None},
  {'module': 'keras.layers',
   'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'units': 256,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'GlorotUniform',
     'config': {'seed': None},
     'registered_name': None},
    'bias_initializer': {'module': 'keras.initializers',
     'class_name': 'Zeros',
     'config': {},
     'registered_name': None},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'registered_name': None,
   'build_config': {'input_shape': (None, 784)}},
  {'module

In [ ]:
# baselineModel = tf.keras.models.load_model('../models/paper/MLP_FASHION_300_100_v2')
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_1.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=40, mlp_epochs=10, mlp_epochs_refit=40, p=1, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True)

In [13]:
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'])
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 3s 5ms/step - loss: 0.4564 - accuracy: 0.8364 - val_loss: 0.3677 - val_accuracy: 0.8650
Epoch 2/40
430/430 [==============================] - 2s 5ms/step - loss: 0.4570 - accuracy: 0.8378 - val_loss: 0.3669 - val_accuracy: 0.8672
Epoch 3/40
430/430 [==============================] - 2s 5ms/step - loss: 0.4571 - accuracy: 0.8363 - val_loss: 0.3688 - val_accuracy: 0.8616
Epoch 4/40
430/430 [==============================] - 2s 5ms/step - loss: 0.4490 - accuracy: 0.8389 - val_loss: 0.3685 - val_accuracy: 0.8632
Epoch 5/40
430/430 [==============================] - 2s 5ms/step - loss: 0.4525 - accuracy: 0.8380 - val_loss: 0.3669 - val_accuracy: 0.8638
Epoch 6/40
430/430 [==============================] - 2s 5ms/step - loss: 0.4504 - accuracy: 0.8400 - val_loss: 0.3652 - val_accuracy: 0.8624
Epoch 7/40
430/430 [==============================] - 2s 5ms/step - loss: 0.4498 - accuracy: 0.8396 - val_loss: 0.3670 - val_accuracy: 0.8634
Epoch 

[0.3853572905063629, 0.8615999817848206]

New Model

In [3]:
activList_0 = np.load('./acivList_Lenet300_Fashion.npy')
activList_1 = np.load('./acivList_Lenet100_Fashion.npy')
activCount_0 = np.load('./acivCount_Lenet300_Fashion.npy')
activCount_1 = np.load('./acivCount_Lenet100_Fashion.npy')

activLists = [activList_0, activList_1]
activCounts = [activCount_0, activCount_1]

In [8]:
# baselineModel = tf.keras.Sequential(
#     [
#         tf.keras.layers.Dense(300, activation="relu", name="layer1"),
#         tf.keras.layers.Dense(100, activation="relu", name="layer2"),
#         tf.keras.layers.Dense(10, activation="softmax", name="layer3"),
#     ]
# )
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, weight_decay=2e-6, nesterov=True)
# baselineModel.compile(
#     optimizer=optimizer, 
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#     metrics=['accuracy'],
#     jit_compile=True)
# baselineModel.build(input_shape=x_train.shape)

In [4]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

In [17]:
# baselineModel.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=128, epochs=200, callbacks=[es])

Epoch 1/200


2023-12-14 11:59:00.176545: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xe8f9c20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-14 11:59:00.176608: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla P40, Compute Capability 6.1
2023-12-14 11:59:00.176623: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla P40, Compute Capability 6.1
2023-12-14 11:59:00.210393: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-14 11:59:00.445084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-12-14 11:59:00.455381: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


 65/430 [===>..........................] - ETA: 0s - loss: 1.0533 - accuracy: 0.6472

2023-12-14 11:59:01.140077: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


430/430 [==============================] - 5s 7ms/step - loss: 0.5549 - accuracy: 0.8076 - val_loss: 0.4028 - val_accuracy: 0.8548
Epoch 2/200
430/430 [==============================] - 1s 2ms/step - loss: 0.3701 - accuracy: 0.8667 - val_loss: 0.3528 - val_accuracy: 0.8698
Epoch 3/200
430/430 [==============================] - 1s 3ms/step - loss: 0.3359 - accuracy: 0.8775 - val_loss: 0.3495 - val_accuracy: 0.8740
Epoch 4/200
430/430 [==============================] - 1s 3ms/step - loss: 0.3125 - accuracy: 0.8852 - val_loss: 0.3330 - val_accuracy: 0.8726
Epoch 5/200
430/430 [==============================] - 1s 3ms/step - loss: 0.2932 - accuracy: 0.8926 - val_loss: 0.3386 - val_accuracy: 0.8756
Epoch 6/200
430/430 [==============================] - 1s 3ms/step - loss: 0.2828 - accuracy: 0.8961 - val_loss: 0.3554 - val_accuracy: 0.8642
Epoch 7/200
430/430 [==============================] - 1s 3ms/step - loss: 0.2679 - accuracy: 0.8994 - val_loss: 0.3354 - val_accuracy: 0.8796
Epoch 8/200

In [18]:
# baselineModel.save('./models/Lenet300100_Fashion')

INFO:tensorflow:Assets written to: ./models/Lenet300100_Fashion/assets


INFO:tensorflow:Assets written to: ./models/Lenet300100_Fashion/assets


In [9]:
baselineModel = tf.keras.models.load_model('./models/Lenet300100_Fashion')

In [10]:
baselineModel.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3979 - accuracy: 0.8918


[0.3978544771671295, 0.8917999863624573]

In [14]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.05, nesterov=True)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_2.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=0, optimizer_config=tf.optimizers.serialize(optimizer), extreme=False, preLoad=False, activLists=activLists, activCounts=activCounts)

Training morphological model...
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min_1 (SoftMaxMin  (55000, 300)              235200    
 )                                                               
                                                                 
 dense_2 (Dense)             (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 11s - loss: 0.9054 - accuracy: 0.7130 - val_loss: 0.5367 - val_accuracy: 0.8070 - 11s/epoch - 6ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4808 - accuracy: 0.8319 - val_loss: 0.4619 - val_accuracy: 0.8342 - 8s/epoch - 5ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4282 - accuracy: 0.8499 - 

In [16]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, nesterov=True)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer='adam', 
            metrics=['accuracy'],
            jit_compile=True)
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 4s 6ms/step - loss: 0.1506 - accuracy: 0.9453 - val_loss: 0.3596 - val_accuracy: 0.8878
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.1476 - accuracy: 0.9465 - val_loss: 0.3853 - val_accuracy: 0.8762
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.1463 - accuracy: 0.9472 - val_loss: 0.3869 - val_accuracy: 0.8834
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.1450 - accuracy: 0.9471 - val_loss: 0.3716 - val_accuracy: 0.8844
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.1426 - accuracy: 0.9482 - val_loss: 0.3678 - val_accuracy: 0.8800
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.1396 - accuracy: 0.9500 - val_loss: 0.3664 - val_accuracy: 0.8838
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.1386 - accuracy: 0.9494 - val_loss: 0.3730 - val_accuracy: 0.8826
Epoch 

[0.41071373224258423, 0.8894000053405762]

p = 1

In [10]:
baselineModel = tf.keras.models.load_model('./models/Lenet300100_Fashion')
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, nesterov=True)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_2.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=1, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True, preLoad=False, activLists=activLists, activCounts=activCounts)

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 300)              235200    
                                                                 
 dense_1 (Dense)             (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 10s - loss: 0.8980 - accuracy: 0.7205 - val_loss: 0.5268 - val_accuracy: 0.8218 - 10s/epoch - 6ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4818 - accuracy: 0.8353 - val_loss: 0.4613 - val_accuracy: 0.8340 - 8s/epoch - 5ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4282 - accuracy: 0.8502 - val_loss: 0.4159 - val_accuracy: 0.8484 - 8s/epoch - 5ms/step
Epoc

In [11]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, nesterov=True)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'],
            jit_compile=True)
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 4s 6ms/step - loss: 0.2386 - accuracy: 0.9120 - val_loss: 0.3316 - val_accuracy: 0.8822
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2348 - accuracy: 0.9136 - val_loss: 0.3312 - val_accuracy: 0.8800
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2337 - accuracy: 0.9141 - val_loss: 0.3318 - val_accuracy: 0.8808
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2330 - accuracy: 0.9141 - val_loss: 0.3307 - val_accuracy: 0.8808
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2326 - accuracy: 0.9148 - val_loss: 0.3307 - val_accuracy: 0.8810
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2324 - accuracy: 0.9148 - val_loss: 0.3315 - val_accuracy: 0.8808
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2322 - accuracy: 0.9153 - val_loss: 0.3303 - val_accuracy: 0.8810
Epoch 

[0.34832221269607544, 0.8781999945640564]

p=0,5

In [30]:
baselineModel = tf.keras.models.load_model('./models/Lenet300100_Fashion')
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, weight_decay=2e-6, nesterov=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_2.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=5, mlp_epochs=10, mlp_epochs_refit=40, p=0.5, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True, preLoad=True, activLists=activLists, activCounts=activCounts)

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 300)              235200    
                                                                 
 dense_1 (Dense)             (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
1719/1719 - 10s - loss: 0.9038 - accuracy: 0.7186 - val_loss: 0.5232 - val_accuracy: 0.8150 - 10s/epoch - 6ms/step
Epoch 2/5
1719/1719 - 8s - loss: 0.4819 - accuracy: 0.8326 - val_loss: 0.4603 - val_accuracy: 0.8400 - 8s/epoch - 4ms/step
Epoch 3/5
1719/1719 - 8s - loss: 0.4282 - accuracy: 0.8501 - val_loss: 0.4200 - val_accuracy: 0.8538 - 8s/epoch - 4ms/step
Epoch 4

In [31]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.05, nesterov=True)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer='adam', 
            metrics=['accuracy'],
            jit_compile=True)
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 4s 7ms/step - loss: 0.3272 - accuracy: 0.8808 - val_loss: 0.3577 - val_accuracy: 0.8676
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3258 - accuracy: 0.8807 - val_loss: 0.3620 - val_accuracy: 0.8668
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3241 - accuracy: 0.8819 - val_loss: 0.3640 - val_accuracy: 0.8696
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3228 - accuracy: 0.8826 - val_loss: 0.3604 - val_accuracy: 0.8670
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3213 - accuracy: 0.8829 - val_loss: 0.3585 - val_accuracy: 0.8674
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3196 - accuracy: 0.8825 - val_loss: 0.3564 - val_accuracy: 0.8670
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3186 - accuracy: 0.8839 - val_loss: 0.3600 - val_accuracy: 0.8698
Epoch 

[0.3764360547065735, 0.8636999726295471]

In [22]:
baselineModel = tf.keras.models.load_model('./models/Lenet300100_Fashion')
optimizer = tf.keras.optimizers.SGD(learning_rate=0.05, nesterov=True)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_2.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=0.5, optimizer_config=tf.optimizers.serialize(optimizer), extreme=True, preLoad=False, activLists=activLists, activCounts=activCounts)

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 300)              235200    
                                                                 
 dense_1 (Dense)             (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 10s - loss: 0.9068 - accuracy: 0.7153 - val_loss: 0.5178 - val_accuracy: 0.8224 - 10s/epoch - 6ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4824 - accuracy: 0.8327 - val_loss: 0.4569 - val_accuracy: 0.8398 - 8s/epoch - 5ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4288 - accuracy: 0.8495 - val_loss: 0.4331 - val_accuracy: 0.8436 - 8s/epoch - 5ms/step
Epoc

In [24]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, nesterov=False)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'],
            jit_compile=True)
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 4s 6ms/step - loss: 0.3031 - accuracy: 0.8895 - val_loss: 0.3686 - val_accuracy: 0.8640
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3015 - accuracy: 0.8891 - val_loss: 0.3460 - val_accuracy: 0.8800
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2996 - accuracy: 0.8902 - val_loss: 0.3747 - val_accuracy: 0.8696
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2979 - accuracy: 0.8913 - val_loss: 0.3753 - val_accuracy: 0.8722
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2970 - accuracy: 0.8915 - val_loss: 0.3484 - val_accuracy: 0.8798
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2972 - accuracy: 0.8910 - val_loss: 0.3418 - val_accuracy: 0.8766
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2958 - accuracy: 0.8925 - val_loss: 0.3481 - val_accuracy: 0.8764
Epoch 

[0.36366477608680725, 0.8708000183105469]

p=0.1

p = [0.5, 0.1]

In [5]:
baselineModel = tf.keras.models.load_model('./models/Lenet300100_Fashion')
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, nesterov=True)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_2.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=0.5, pList=[0.5, 0.05], optimizer_config=tf.optimizers.serialize(optimizer), extreme=True, preLoad=True, activLists=activLists, activCounts=activCounts)

2023-12-29 10:33:37.543666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2031] Ignoring visible gpu device (device: 2, name: NVIDIA GeForce GT 1030, pci bus id: 0000:84:00.0, compute capability: 6.1) with core count: 3. The minimum required count is 8. You can adjust this requirement with the env var TF_MIN_GPU_MULTIPROCESSOR_COUNT.
2023-12-29 10:33:38.894310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23664 MB memory:  -> device: 0, name: Tesla P40, pci bus id: 0000:03:00.0, compute capability: 6.1
2023-12-29 10:33:38.894958: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 23664 MB memory:  -> device: 1, name: Tesla P40, pci bus id: 0000:04:00.0, compute capability: 6.1


Training morphological model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 300)              235200    
                                                                 
 dense (Dense)               (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30


2023-12-29 10:33:40.670083: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x221a8540 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-29 10:33:40.670153: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla P40, Compute Capability 6.1
2023-12-29 10:33:40.670169: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla P40, Compute Capability 6.1
2023-12-29 10:33:40.685603: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-29 10:33:41.172766: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-12-29 10:33:41.292692: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-29 10:33:41.494044: I ./tensorflow/compiler/jit/device_compiler.h:186] Compil

1719/1719 - 12s - loss: 0.8957 - accuracy: 0.7152 - val_loss: 0.5143 - val_accuracy: 0.8240 - 12s/epoch - 7ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4815 - accuracy: 0.8313 - val_loss: 0.4572 - val_accuracy: 0.8404 - 8s/epoch - 4ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4291 - accuracy: 0.8495 - val_loss: 0.4329 - val_accuracy: 0.8456 - 8s/epoch - 4ms/step
Epoch 4/30
1719/1719 - 8s - loss: 0.3969 - accuracy: 0.8603 - val_loss: 0.3970 - val_accuracy: 0.8578 - 8s/epoch - 4ms/step
Epoch 5/30
1719/1719 - 8s - loss: 0.3751 - accuracy: 0.8680 - val_loss: 0.3883 - val_accuracy: 0.8626 - 8s/epoch - 4ms/step
Epoch 6/30
1719/1719 - 8s - loss: 0.3577 - accuracy: 0.8750 - val_loss: 0.3782 - val_accuracy: 0.8644 - 8s/epoch - 4ms/step
Epoch 7/30
1719/1719 - 8s - loss: 0.3455 - accuracy: 0.8783 - val_loss: 0.3843 - val_accuracy: 0.8614 - 8s/epoch - 4ms/step
Epoch 8/30
1719/1719 - 8s - loss: 0.3332 - accuracy: 0.8832 - val_loss: 0.3515 - val_accuracy: 0.8750 - 8s/epoch - 4ms/step
Epoch 9/30
1719/1

In [6]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'],
            jit_compile=True)
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 4s 6ms/step - loss: 0.2899 - accuracy: 0.8931 - val_loss: 0.3561 - val_accuracy: 0.8730
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2869 - accuracy: 0.8935 - val_loss: 0.3554 - val_accuracy: 0.8706
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2859 - accuracy: 0.8939 - val_loss: 0.3549 - val_accuracy: 0.8714
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2854 - accuracy: 0.8935 - val_loss: 0.3545 - val_accuracy: 0.8730
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2850 - accuracy: 0.8942 - val_loss: 0.3554 - val_accuracy: 0.8738
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2848 - accuracy: 0.8937 - val_loss: 0.3549 - val_accuracy: 0.8728
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2844 - accuracy: 0.8943 - val_loss: 0.3540 - val_accuracy: 0.8746
Epoch 

[0.3687502443790436, 0.8693000078201294]

p=[5% 0.1%]

In [8]:
baselineModel = tf.keras.models.load_model('./models/Lenet300100_Fashion')
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, nesterov=True)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_2.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=0.5, pList=[5, 0.1], optimizer_config=tf.optimizers.serialize(optimizer), extreme=True, preLoad=False, activLists=activLists, activCounts=activCounts)

Training morphological model...
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min_1 (SoftMaxMin  (55000, 300)              235200    
 )                                                               
                                                                 
 dense_2 (Dense)             (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 10s - loss: 0.8924 - accuracy: 0.7215 - val_loss: 0.5241 - val_accuracy: 0.8194 - 10s/epoch - 6ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4798 - accuracy: 0.8333 - val_loss: 0.4489 - val_accuracy: 0.8426 - 8s/epoch - 5ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4269 - accuracy: 0.8521 - 

In [9]:
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, nesterov=True)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'],
            jit_compile=True)
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 4s 7ms/step - loss: 0.2630 - accuracy: 0.9044 - val_loss: 0.3295 - val_accuracy: 0.8846
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2590 - accuracy: 0.9065 - val_loss: 0.3291 - val_accuracy: 0.8812
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2579 - accuracy: 0.9065 - val_loss: 0.3272 - val_accuracy: 0.8838
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2571 - accuracy: 0.9072 - val_loss: 0.3267 - val_accuracy: 0.8834
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2567 - accuracy: 0.9074 - val_loss: 0.3275 - val_accuracy: 0.8830
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2564 - accuracy: 0.9075 - val_loss: 0.3268 - val_accuracy: 0.8826
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2562 - accuracy: 0.9079 - val_loss: 0.3262 - val_accuracy: 0.8826
Epoch 

[0.3572413921356201, 0.8748999834060669]

p=[3% 0.1%]

In [11]:
baselineModel = tf.keras.models.load_model('./models/Lenet300100_Fashion')
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, nesterov=True)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_2.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=0.5, pList=[3, 0.1], optimizer_config=tf.optimizers.serialize(optimizer), extreme=True, preLoad=False, activLists=activLists, activCounts=activCounts)

Training morphological model...
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min_1 (SoftMaxMin  (55000, 300)              235200    
 )                                                               
                                                                 
 dense_2 (Dense)             (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 10s - loss: 0.8970 - accuracy: 0.7223 - val_loss: 0.5136 - val_accuracy: 0.8196 - 10s/epoch - 6ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4834 - accuracy: 0.8307 - val_loss: 0.4445 - val_accuracy: 0.8416 - 8s/epoch - 5ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4287 - accuracy: 0.8490 - 

In [12]:
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, nesterov=True)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'],
            jit_compile=True)
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)
# 0.8611000180244446

Epoch 1/40
430/430 [==============================] - 3s 6ms/step - loss: 0.2414 - accuracy: 0.9101 - val_loss: 0.3167 - val_accuracy: 0.8832
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2376 - accuracy: 0.9122 - val_loss: 0.3156 - val_accuracy: 0.8834
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2365 - accuracy: 0.9131 - val_loss: 0.3144 - val_accuracy: 0.8838
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2359 - accuracy: 0.9133 - val_loss: 0.3140 - val_accuracy: 0.8854
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2356 - accuracy: 0.9132 - val_loss: 0.3139 - val_accuracy: 0.8852
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2353 - accuracy: 0.9131 - val_loss: 0.3141 - val_accuracy: 0.8854
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.2351 - accuracy: 0.9133 - val_loss: 0.3138 - val_accuracy: 0.8848
Epoch 

[0.3557592034339905, 0.8810999989509583]

In [14]:
pruned.save('./models/Lenet300100_Fashion_pruned_3/')

INFO:tensorflow:Assets written to: ./models/Lenet300100_Fashion_pruned_3/assets


INFO:tensorflow:Assets written to: ./models/Lenet300100_Fashion_pruned_3/assets


p=[3 0]%

In [9]:
baselineModel = tf.keras.models.load_model('./models/Lenet300100_Fashion')
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, nesterov=True)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_2.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=5, mlp_epochs=10, mlp_epochs_refit=40, p=0.5, pList=[3, 0], optimizer_config=tf.optimizers.serialize(optimizer), extreme=True, preLoad=True, activLists=activLists, activCounts=activCounts)

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 300)              235200    
                                                                 
 dense_1 (Dense)             (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
1719/1719 - 10s - loss: 0.9032 - accuracy: 0.7139 - val_loss: 0.5225 - val_accuracy: 0.8212 - 10s/epoch - 6ms/step
Epoch 2/5
1719/1719 - 8s - loss: 0.4837 - accuracy: 0.8309 - val_loss: 0.4722 - val_accuracy: 0.8354 - 8s/epoch - 4ms/step
Epoch 3/5
1719/1719 - 8s - loss: 0.4310 - accuracy: 0.8498 - val_loss: 0.4129 - val_accuracy: 0.8558 - 8s/epoch - 4ms/step
Epoch 4

In [11]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, nesterov=True)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer='adam', 
            metrics=['accuracy'],
            jit_compile=True)
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 4s 7ms/step - loss: 0.3208 - accuracy: 0.8822 - val_loss: 0.3795 - val_accuracy: 0.8610
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3207 - accuracy: 0.8819 - val_loss: 0.3824 - val_accuracy: 0.8572
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3202 - accuracy: 0.8826 - val_loss: 0.3802 - val_accuracy: 0.8584
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3205 - accuracy: 0.8823 - val_loss: 0.3807 - val_accuracy: 0.8620
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3199 - accuracy: 0.8825 - val_loss: 0.3795 - val_accuracy: 0.8598
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3193 - accuracy: 0.8827 - val_loss: 0.3825 - val_accuracy: 0.8602
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.3196 - accuracy: 0.8828 - val_loss: 0.3798 - val_accuracy: 0.8570
Epoch 

[0.3901026248931885, 0.8610000014305115]

p=0,01%

In [7]:
baselineModel = tf.keras.models.load_model('./models/Lenet300100_Fashion')
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, nesterov=True)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_2.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=0.01, pList=[], optimizer_config=tf.optimizers.serialize(optimizer), extreme=True, preLoad=False, activLists=activLists, activCounts=activCounts)

Training morphological model...
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min_2 (SoftMaxMin  (55000, 300)              235200    
 )                                                               
                                                                 
 dense_2 (Dense)             (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 10s - loss: 0.9089 - accuracy: 0.7166 - val_loss: 0.5223 - val_accuracy: 0.8184 - 10s/epoch - 6ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4858 - accuracy: 0.8311 - val_loss: 0.4469 - val_accuracy: 0.8478 - 8s/epoch - 5ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4340 - accuracy: 0.8488 - 

In [9]:
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, nesterov=True)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'],
            jit_compile=True)
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 4s 6ms/step - loss: 0.0793 - accuracy: 0.9725 - val_loss: 0.5413 - val_accuracy: 0.8784
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0698 - accuracy: 0.9766 - val_loss: 0.5387 - val_accuracy: 0.8790
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0668 - accuracy: 0.9783 - val_loss: 0.5377 - val_accuracy: 0.8802
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0652 - accuracy: 0.9793 - val_loss: 0.5375 - val_accuracy: 0.8812
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0643 - accuracy: 0.9797 - val_loss: 0.5373 - val_accuracy: 0.8812
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0637 - accuracy: 0.9802 - val_loss: 0.5370 - val_accuracy: 0.8800
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0630 - accuracy: 0.9805 - val_loss: 0.5372 - val_accuracy: 0.8812
Epoch 

[0.5396414399147034, 0.8804000020027161]

p=[0.05 0.01]

In [15]:
baselineModel = tf.keras.models.load_model('./models/Lenet300100_Fashion')
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.975, nesterov=True)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
pruned, pruned_hp1, pruned_ep1 = pruning_tool_MLP_v4_2.pruning_MLP(baselineModel, x_train, y_train, x_val, y_val, x_test, y_test, batch_size=128, morph_epochs=30, mlp_epochs=10, mlp_epochs_refit=40, p=0.01, pList=[0.01, 0.0], optimizer_config=tf.optimizers.serialize(optimizer), extreme=True, preLoad=False, activLists=activLists, activCounts=activCounts)

Training morphological model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 soft_max_min (SoftMaxMin)   (55000, 300)              235200    
                                                                 
 dense_1 (Dense)             (55000, 10)               3000      
                                                                 
Total params: 238200 (930.47 KB)
Trainable params: 238200 (930.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1719/1719 - 10s - loss: 0.9030 - accuracy: 0.7203 - val_loss: 0.5305 - val_accuracy: 0.8168 - 10s/epoch - 6ms/step
Epoch 2/30
1719/1719 - 8s - loss: 0.4840 - accuracy: 0.8319 - val_loss: 0.4550 - val_accuracy: 0.8400 - 8s/epoch - 4ms/step
Epoch 3/30
1719/1719 - 8s - loss: 0.4301 - accuracy: 0.8498 - val_loss: 0.4249 - val_accuracy: 0.8484 - 8s/epoch - 4ms/step
Epoc

In [20]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, nesterov=True)
pruned.compile(loss='sparse_categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy'],
            jit_compile=True)
pruned.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=40, batch_size=128, callbacks=[es])
pruned.evaluate(x_test, y_test)

Epoch 1/40
430/430 [==============================] - 3s 6ms/step - loss: 0.0491 - accuracy: 0.9868 - val_loss: 0.5540 - val_accuracy: 0.8766
Epoch 2/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0490 - accuracy: 0.9866 - val_loss: 0.5546 - val_accuracy: 0.8772
Epoch 3/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0489 - accuracy: 0.9868 - val_loss: 0.5563 - val_accuracy: 0.8772
Epoch 4/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0488 - accuracy: 0.9867 - val_loss: 0.5576 - val_accuracy: 0.8772
Epoch 5/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0487 - accuracy: 0.9869 - val_loss: 0.5566 - val_accuracy: 0.8782
Epoch 6/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0487 - accuracy: 0.9871 - val_loss: 0.5564 - val_accuracy: 0.8772
Epoch 7/40
430/430 [==============================] - 1s 3ms/step - loss: 0.0486 - accuracy: 0.9866 - val_loss: 0.5572 - val_accuracy: 0.8778
Epoch 

[0.5682822465896606, 0.880299985408783]

: 